# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import logging

import pandas as pd

import core.config.config_ as cconconf
import helpers.hdbg as hdbg
import helpers.hprint as hprint
import research_amp.transform as ramptran

In [ ]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

hprint.config_notebook()

# Config

In [ ]:
def get_CrossSectionalLearning_config() -> cconconf.Config:
    """
    Get config, that specifies params for getting raw data from `crypto
    chassis`.
    """
    config = cconconf.Config()
    # Load parameters.
    # config.add_subconfig("load")
    # Data parameters.
    config.add_subconfig("data")
    config["data"]["full_symbols"] = [
        "binance::ADA_USDT",
        "binance::BNB_USDT",
        "binance::BTC_USDT",
        "binance::DOGE_USDT",
        "binance::EOS_USDT",
        "binance::ETH_USDT",
        "binance::SOL_USDT",
        "binance::XRP_USDT",
        "binance::LUNA_USDT",
        "binance::DOT_USDT",
        "binance::LTC_USDT",
        "binance::UNI_USDT",
    ]
    config["data"]["start_date"] = pd.Timestamp("2022-01-01", tz="UTC")
    config["data"]["end_date"] = pd.Timestamp("2022-02-01", tz="UTC")
    # Transformation parameters.
    config.add_subconfig("transform")
    config["transform"]["resampling_rule"] = "1T"
    config["transform"]["rets_type"] = "pct_change"
    # Analysis parameters.
    config.add_subconfig("analysis")
    config["analysis"][
        "reference_rets"
    ] = "close.ret_0"  # e.g., "vwap.ret_0", "twap.ret_0"
    return config

In [ ]:
config = get_CrossSectionalLearning_config()
print(config)

# Load the data

Specs for the current data snapshot:
- Data type: `OHLCV`
- Universe: `v5` (excl. missing coins, see `research_amp/cc/notebooks/master_tradability_analysis.ipynb` for reference)
- Data range: January 2022

In [ ]:
# TODO(Max): Refactor the loading part once #1766 is implemented.

# Read from crypto_chassis directly.
# full_symbols = config["data"]["full_symbols"]
# start_date = config["data"]["start_date"]
# end_date = config["data"]["end_date"]
# ohlcv_cc = raccchap.read_crypto_chassis_ohlcv(full_symbols, start_date, end_date)

# Read saved 1 month of data.
ohlcv_cc = pd.read_csv("/shared_data/ohlcv_cc_v5.csv", index_col="timestamp")
ohlcv_cc.index = pd.to_datetime(ohlcv_cc.index)
ohlcv_cols = [
    "open",
    "high",
    "low",
    "close",
    "volume",
    "full_symbol",
]
ohlcv_cc = ohlcv_cc[ohlcv_cols]
ohlcv_cc.head(3)

# Compute returns

In [ ]:
# VWAP, TWAP transformation.
resampling_rule = config["transform"]["resampling_rule"]
df = ramptran.calculate_vwap_twap(ohlcv_cc, resampling_rule)
# Returns calculation.
rets_type = config["transform"]["rets_type"]
df = ramptran.calculate_returns(df, rets_type)
# Choose reference returns to proceed to further analysis.
df = df[[config["analysis"]["reference_rets"]]]
df.head(3)